In [59]:
import os 
import json
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

# Fixed system message variable name
system_message = "You are a helpful assistant for movies recommendation and also book movie shows. You will use tools for that. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "You are actually slightly teasing."

load_dotenv(override=True)
openai_key = os.getenv("OPENAI_API_KEY")
if openai_key:
    print("key found")
else:
    print("error in finding the key")

MODEL = "gpt-4o-mini"
openai = OpenAI()

movie_list = {
    "sci_fi": [
        "Inception",
        "Interstellar",
        "The Matrix",
        "Avatar",
        "Dune",
        "Dune: Part Two",
        "Doctor Strange",
        "Doctor Strange in the Multiverse of Madness",
        "Guardians of the Galaxy",
        "Guardians of the Galaxy Vol. 2",
        "Iron Man",
        "Iron Man 2",
        "Iron Man 3",
        "Spider-Man: No Way Home",
        "Avengers: Infinity War",
        "Avengers: Endgame",
        "Black Panther"
    ],
    "action": [
        "The Dark Knight",
        "John Wick",
        "John Wick 2",
        "John Wick 3",
        "Gladiator",
        "Iron Man",
        "Iron Man 2",
        "Iron Man 3",
        "Guardians of the Galaxy",
        "Guardians of the Galaxy Vol. 2",
        "Spider-Man: No Way Home",
        "Avengers: Infinity War",
        "Avengers: Endgame",
        "Black Panther"
    ],
    "drama": [
        "Titanic",
        "The Godfather",
        "The Godfather Part II",
        "Forrest Gump",
        "The Shawshank Redemption",
        "Oppenheimer",
        "Fight Club"
    ],
    "crime": [
        "Pulp Fiction",
        "The Godfather",
        "The Godfather Part II",
        "The Dark Knight",
        "Fight Club"
    ],
    "adventure": [
        "Avatar",
        "Gladiator",
        "Dune",
        "Dune: Part Two",
        "Guardians of the Galaxy",
        "Guardians of the Galaxy Vol. 2",
        "Spider-Man: No Way Home",
        "Avengers: Infinity War",
        "Avengers: Endgame",
        "Black Panther"
    ],
    "romance": [
        "Titanic",
        "Forrest Gump",
        "Avatar"
    ]
}

def recommend_movie(genre: str):
    print(f"Tool recommend_movie called for {genre}")
    if genre in movie_list:
        return {
            "recommended_movies": movie_list[genre]
        }
    else:
        return {
            "error": f"Sorry, genre '{genre}' is not available."
        }

movie_recommended_tool = {
    "type": "function",
    "function": {
        "name": "recommend_movie",
        "description": "Recommends a list of movies based on a user-provided genre.",
        "parameters": {
            "type": "object",
            "properties": {
                "genre": {
                    "type": "string",
                    "description": "The movie genre to recommend (e.g., 'action', 'comedy', 'sci_fi', 'drama', 'crime', 'adventure', 'romance')."
                }
            },
            "required": ["genre"]
        }
    }
}

# Movie showtime dataset
movie_showtimes = {
    "Inception": ["1:00 PM", "4:00 PM", "8:00 PM"],
    "Interstellar": ["2:00 PM", "6:00 PM", "9:30 PM"],
    "The Dark Knight": ["12:30 PM", "3:45 PM", "7:15 PM", "10:00 PM"],
    "Oppenheimer": ["1:15 PM", "5:00 PM", "9:00 PM"],
    "Avatar": ["12:00 PM", "4:15 PM", "8:30 PM"],
    "Titanic": ["11:00 AM", "3:00 PM", "7:00 PM"],
    "The Matrix": ["1:45 PM", "5:15 PM", "8:45 PM"],
    "John Wick": ["2:00 PM", "6:00 PM", "9:00 PM"],
    "John Wick 2": ["12:45 PM", "4:00 PM", "7:30 PM", "10:15 PM"],
    "John Wick 3": ["1:30 PM", "5:00 PM", "8:15 PM"],
    "The Godfather": ["12:00 PM", "3:30 PM", "7:00 PM"],
    "The Godfather Part II": ["1:00 PM", "4:30 PM", "8:00 PM"],
    "Pulp Fiction": ["2:15 PM", "5:30 PM", "9:00 PM"],
    "Fight Club": ["1:10 PM", "4:40 PM", "8:20 PM"],
    "Forrest Gump": ["12:30 PM", "3:50 PM", "7:15 PM"],
    "The Shawshank Redemption": ["1:25 PM", "5:00 PM", "8:45 PM"],
    "Gladiator": ["12:15 PM", "3:45 PM", "7:30 PM"],
    "Dune": ["1:20 PM", "4:50 PM", "8:10 PM"],
    "Dune: Part Two": ["12:45 PM", "4:15 PM", "7:45 PM"],
    "Spider-Man: No Way Home": ["2:10 PM", "5:40 PM", "9:15 PM"],
    "Avengers: Endgame": ["12:00 PM", "3:30 PM", "7:15 PM", "10:30 PM"],
    "Avengers: Infinity War": ["1:00 PM", "4:20 PM", "8:00 PM"],
    "Doctor Strange": ["12:25 PM", "3:55 PM", "7:20 PM"],
    "Doctor Strange in the Multiverse of Madness": ["1:30 PM", "5:00 PM", "8:30 PM"],
    "Black Panther": ["12:10 PM", "3:40 PM", "7:00 PM"],
    "Iron Man": ["2:00 PM", "5:20 PM", "8:50 PM"],
    "Iron Man 2": ["1:15 PM", "4:35 PM", "8:05 PM"],
    "Iron Man 3": ["12:50 PM", "4:10 PM", "7:40 PM"],
    "Guardians of the Galaxy": ["1:40 PM", "5:10 PM", "8:40 PM"],
    "Guardians of the Galaxy Vol. 2": ["2:05 PM", "5:35 PM", "9:05 PM"],
    "me before you": ["2:05 PM", "5:35 PM", "9:05 PM"]
}

def get_movie_showtimes(movie_title: str):
    movie_title = movie_title.strip()
    if movie_title in movie_showtimes:
        return {
            "movie": movie_title,
            "showtimes": movie_showtimes[movie_title]
        }
    else:
        return {
            "error": f"Sorry, '{movie_title}' is not currently playing."
        }

# Fixed showtime tool definition
showtime_tool = {
    "type": "function",
    "function": {
        "name": "get_movie_showtimes",
        "description": "Get showtimes for a specific movie.",
        "parameters": {
            "type": "object",
            "properties": {
                "movie_title": {
                    "type": "string",
                    "description": "The title of the movie to get showtimes for."
                }
            },
            "required": ["movie_title"]
        }
    }
}

# Define all tools
tools = [movie_recommended_tool, showtime_tool]

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    
    if tool_call.function.name == "recommend_movie":
        genre = arguments.get('genre')
        result = recommend_movie(genre)
    elif tool_call.function.name == "get_movie_showtimes":
        movie_title = arguments.get('movie_title')
        result = get_movie_showtimes(movie_title)
    
    response = {
        "role": "tool",
        "content": json.dumps(result),
        "tool_call_id": tool_call.id
    }
    return response

def chat(message, history):
    # Convert Gradio history format to OpenAI format
    messages = [{"role": "system", "content": system_message}]
    
    # When type="messages", history is already in OpenAI format
    messages.extend(history)
    
    messages.append({"role": "user", "content": message})
    
    response = openai.chat.completions.create(
        model=MODEL, 
        messages=messages, 
        tools=tools
    )
    
    if response.choices[0].finish_reason == "tool_calls":
        message_with_tool = response.choices[0].message
        tool_response = handle_tool_call(message_with_tool)
        
        messages.append({
            "role": "assistant",
            "content": message_with_tool.content,
            "tool_calls": message_with_tool.tool_calls
        })
        messages.append(tool_response)
        
        response = openai.chat.completions.create(
            model=MODEL, 
            messages=messages
        )
    
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

key found
* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Tool recommend_movie called for romance
